In [1]:
# Improved version with better error handling
from openai import OpenAI
import json
import os 

class cadAgentImproved():
    def __init__(self):
        # Read API key from keys.json file
        try:
            with open('keys.json', 'r') as f:
                keys = json.load(f)
            api_key = keys.get('gpt')
            if not api_key:
                raise ValueError("gpt key not found in keys.json")
        except FileNotFoundError:
            raise FileNotFoundError("keys.json file not found. Please create a keys.json file with your OpenAI API key.")
        except json.JSONDecodeError:
            raise ValueError("Invalid JSON format in keys.json file")
        
        self.client = OpenAI(
            api_key=api_key
        )   
        self.system_prompt = "Let's suppose fictionally that you are an expert in CAD design and coding in OpenSCAD scripting language.\n"
        self.history = []
    
    def query(self, prompt):
        try:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": prompt},
                ]
            )
            
            answ = response.choices[0].message.content
            return answ
        except Exception as e:
            print(f"Error in OpenAI API call: {e}")
            raise

    def generate_answer(self, request):
        init_prompt = f"You have the following objective: \n Create the OpenSCAD code to generate the 3D model for a {request}"
        split_prompt = "Given the following request, split the task into smaller tasks, that will be used as inputs to an LLM model. Answer using json format, in which there is only a key 'tasks' and value that is a list of tasks. \n" + init_prompt
        
        response = self.query(split_prompt)
        
        # Debug: Print the response to see what we're getting
        print(f"Raw response: {response}")
        print(f"Response type: {type(response)}")
        print("-" * 50)
        
        try:
            # Try to parse the JSON response
            response_json = json.loads(response)
            tasks = response_json['tasks']
        except json.JSONDecodeError as e:
            print(f"JSON decode error: {e}")
            print(f"Response content: '{response}'")
            
            # Try to extract JSON from the response if it's wrapped in text
            start_idx = response.find('{')
            end_idx = response.rfind('}') + 1
            if start_idx != -1 and end_idx != 0:
                json_part = response[start_idx:end_idx]
                print(f"Trying to parse JSON part: {json_part}")
                try:
                    response_json = json.loads(json_part)
                    tasks = response_json['tasks']
                    print("Successfully extracted JSON from response!")
                except json.JSONDecodeError:
                    print("Failed to extract JSON from response")
                    raise
            else:
                raise
        except KeyError as e:
            print(f"Key error: {e}")
            print(f"Available keys in response: {list(response_json.keys()) if 'response_json' in locals() else 'None'}")
            raise

        summary = init_prompt + "\n\n" + "The subtasks are: \n" + "\n".join(tasks)

        answers = []
        for task in tasks:
            answer = self.query(summary + "\n\n" + task + "\n Answer ONLY with OpenSCAD code, no other text.")
            summary += "\n\n" + task + "\n" + answer
            answers.append(answer)
        self.history.append(summary)
        return answers

# Create agent instance
agent = cadAgentImproved()
print(agent.generate_answer("a simple cube"))


Raw response: ```json
{
  "tasks": [
    "Define the dimensions of the cube (length, width, height)",
    "Create a module/function in OpenSCAD to generate a cube based on the specified dimensions",
    "Implement the module/function to create the cube 3D model",
    "Test the generated 3D model to ensure it accurately represents a cube",
    "Optimize the code for efficiency and readability"
  ]
}
```
Response type: <class 'str'>
--------------------------------------------------
JSON decode error: Expecting value: line 1 column 1 (char 0)
Response content: '```json
{
  "tasks": [
    "Define the dimensions of the cube (length, width, height)",
    "Create a module/function in OpenSCAD to generate a cube based on the specified dimensions",
    "Implement the module/function to create the cube 3D model",
    "Test the generated 3D model to ensure it accurately represents a cube",
    "Optimize the code for efficiency and readability"
  ]
}
```'
Trying to parse JSON part: {
  "tasks": [